In [1]:
import os

os.environ["OPENAI_API_KEY"] =


In [2]:
!pip install datasets deepeval python-dotenv

  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 504.1/504.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 7.6 MB/s 

In [4]:
# Создал свою метрику креативности анекдота
!python evaluation_openai.py --num_samples 30

============== GT EXAMPLES ===============
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': '- Слышали, коллега, в пятницу двое российских ученых приняли вакцину от короновируса?'}, {'role': 'assistant', 'content': 'Ничего удивительного, до самоизоляции у нас вся страна в пятницу эту вакцину на работе принимала.', 'weight': 1}]}
-----------
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': 'Утром муж за завтраком говорит жене:'}, {'role': 'assistant', 'content': 'Надо бы в инет слазить, узнать,

In [5]:
# Промт 1 лучше соответствует критерию "стиля", 0.388 pvalue=8e-5, что означает сильное отличие от 0.5
# Промт 2 работает значительно хуже по  критерию "стиля". 0.261 pvalue=7.8e-11, что означает сильное отличие от 0.5
# Промт 1 лучше соответствует критерию "креативности", 0.678 pvalue=4.5e-08, что означает сильное отличие от 0.5
# Промт 2 работает чуть хуже по критерию "креативности", 0.597 pvalue=0.0019, что означает сильное отличие от 0.5

In [6]:
# Дообучим модель в openai

In [7]:
from openai import OpenAI

client = OpenAI()

# Загружаем файл в openai и передаем file_id дальше
client.files.create(file=open("jokes.jsonl", "rb"), purpose="fine-tune")


FileObject(id='file-NRLhgiFCmH7FDGG7XBLvP6', bytes=21139, created_at=1734624462, filename='jokes.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [8]:
# Создаем задачу на файн тюнинг с датасетом file-NRLhgiFCmH7FDGG7XBLvP6
client.fine_tuning.jobs.create(
    training_file="file-NRLhgiFCmH7FDGG7XBLvP6", model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-B7beTZ3trvzR4LCqZjLstQgX', created_at=1734624483, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=[], seed=232243316, status='validating_files', trained_tokens=None, training_file='file-NRLhgiFCmH7FDGG7XBLvP6', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None, method={'type': 'supervised', 'supervised': {'hyperparameters': {'batch_size': 'auto', 'learning_rate_multiplier': 'auto', 'n_epochs': 'auto'}}})

In [27]:
# Проверяем статус задачи
print(client.fine_tuning.jobs.retrieve("ftjob-B7beTZ3trvzR4LCqZjLstQgX").status)
print(
    client.fine_tuning.jobs.retrieve("ftjob-B7beTZ3trvzR4LCqZjLstQgX").fine_tuned_model
)

succeeded
ft:gpt-4o-mini-2024-07-18:personal::AgDTrkNt


In [28]:
# Указывю эту модель в файле в методе get_model_predictions
!python evaluation_openai.py --num_samples 30

============== GT EXAMPLES ===============
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'assistant', 'content': '- Мне кажется люди заигрались с этим очеловечиванием животных.'}, {'role': 'user', 'content': 'Слушай, давай уже не будем о неграх.'}, {'role': 'assistant', 'content': 'Вообще-то я о кошках.', 'weight': 1}]}
-----------
{'messages': [{'role': 'system', 'content': 'Ты чат бот, который генерирует анекдоты. Продолжи диалог одной репликой так, чтоб получился анекдот. Example input: - Я не могу выйти на работу, потому что жена сломала два ребра. - А при чём тут ты? Expected output: - Мои!'}, {'role': 'user', 'content': 'Конец 70-х, самолет Москва - Париж. Незадолго перед посадкой в салон входит стюардесса и говорит:'}, {'role': 'assistant', 'conten

In [ ]:
# Промт 1 лучше соответствует критерию "стиля", 0.4638 pvalue=0.284, что означает не сильное отличие от 0.5
# Промт 2 работает значительно хуже по  критерию "стиля". 0.3879 pvalue=0.0022, что означает сильное отличие от 0.5
# Промт 1 работает чуть хуже по критерию "креативности", 0.5370 pvalue=0.269, что означает не сильное отличие от 0.5
# Промт 2 лучше соответствует критерию "креативности", 0.5587 pvalue=0.0639, что означает не сильное отличие от 0.5

# В среднем, будто затюненная модель отработала чуть хуже т.к метрика по стилю стала выше, а по креативности сильно не упала